In [1]:
# 라이브러리 로드 
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

In [ ]:
# 데이터 로드
df = pd.read_csv("../data/ratings_train.txt", sep='\t')

In [3]:
# 데이터 결측치를 제외 
df.dropna(inplace=True)

In [7]:
# 독립 변수, 종속 변수 생성
X = df['document'][:5000]
Y = df['label'][:5000]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=42
)

In [10]:
print(X_train.shape)

(4000,)


- 파이프라인 생성 
    - 백터화(TF-IDF 방식)
        - 토큰화 함수(Komoran 형태소 방식) 포함 
    - LSA
        - TruncatedSVD() 차원 축소를 사용
    - Scaler
        - StandardScaler를 이용
    - Model
        - SVC() 모델 

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
# 토큰화 함수 
komoran = Komoran()
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG']

def tokenize(text):
    result = []
    for word, pos in komoran.pos(text):
        if pos in allow_pos:
            result.append(word)
    return result

# 백터화 객체 생성
vectorizer = TfidfVectorizer(
    # 토큰화 방식 
    tokenizer= tokenize, 
    lowercase= False, 
    min_df = 5, 
    max_df= 0.8, 
    ngram_range=(1, 1)
)

In [13]:
# 차원 축소 객체 생성 
svd = TruncatedSVD(
    n_components= 200, 
    random_state=42
)

In [14]:
# StandardScaler 
scaler = StandardScaler(with_mean=True)

In [15]:
svc = SVC(
    kernel= 'linear', 
    random_state=42, 
    probability=False
)

In [16]:
# 위의 4개의 작업 을 하나의 객체로 연결 
pipe = Pipeline(
  [
      ('vector', vectorizer), 
      ('lsa', svd), 
      ('scaler', scaler), 
      ('model', svc)
  ]  
)

In [24]:
# 최적의 매개변수를 찾기 위해 매개변수들을 지정 
# pipe를 이용하게 되면 pipe step의 객체들의 key__parameter : [val1, val2]
grid_params = {
    "vector__ngram_range" : [ (1,1), (1,2) ],
    "lsa__n_components" : [100, 200], 
    "model__C" : [1.0, 2.0]
}

In [25]:
# GridSearchCV를 이용하여 각각 매개변수들을 대입하여 
# 최적의 매개변수를 찾는다
grid = GridSearchCV(
    estimator= pipe, 
    param_grid= grid_params, 
    cv = 2, 
    scoring='accuracy', 
    verbose=1, 
    # n_jobs=-1   # remote error 발생 시 주석 처리 
)

In [ ]:
grid.fit(X_train, Y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:517: Use